In [2]:
from openai import OpenAI
import os
import json
from dotenv import load_dotenv
from pathlib import Path
from pydantic import BaseModel

extracted_answers = []
query_file = "queries.json"


dotenv_path = Path('../../global_environment.env')
load_dotenv(dotenv_path=dotenv_path)
openai_key = os.getenv('OPENAI_KEY')

client = OpenAI(
        api_key=openai_key
)

if(openai_key):
    print('Key loaded successfully')
else:
    print('API Key not found')

Key loaded successfully


In [8]:
class SignalRating(BaseModel):
    signal_id: str
    smoothness_score: float
    smoothness_keywords: list[str]
    roughness_keywords: list[str]

class SignalKeywords(BaseModel):
    signal_id: str
    sensory_keywords: list[str]
    emotional_keywords: list[str]
    metaphor_keywords: list[str]

description_file = '../../Data/signal-descriptions/collated_descriptions.json'
queries_file = '../queries.json'
with open(description_file, 'r') as f:
    descriptionsJson = json.load(f)
with open(queries_file, 'r') as f:
    queries = json.load(f)

def getRating(signalObj):
    signal_id = signalObj['signal_index']
    print("Extracting ratings for signal index ", signal_id)
    descriptions = [*signalObj['sensory_descriptions'], *signalObj['emotional_descriptions'], *signalObj['associative_descriptions']]
    descriptionList = '\n'.join(descriptions)
    response = client.beta.chat.completions.parse(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are an assistant that processes descriptions of haptic feedback and returns json objects"},
                {"role": "user", "content": f"signal id: F{signal_id}\n"+queries['smoothnessRatingPrompt']+'\n'+descriptionList},
                ],
                response_format=SignalRating
        )
    print(response.choices[0].message.content)
    return json.loads(response.choices[0].message.content) # response.choices[0].message.content

def getKeywords(signalObj):
    signal_id = signalObj['signal_index']
    print("Extracting keywords for signal index ", signal_id)
    descriptions = [*signalObj['sensory_descriptions'], *signalObj['emotional_descriptions'], *signalObj['associative_descriptions']]
    descriptionList = '\n'.join(descriptions)
    response = client.beta.chat.completions.parse(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are an assistant that processes descriptions of haptic feedback and returns json objects"},
                {"role": "user", "content": f"signal id: F{signal_id}\n"+queries['extractionPrompt']+'\n'+descriptionList},
                ],
                response_format=SignalKeywords
        )
    #print(response.choices[0].message.content)
    return json.loads(response.choices[0].message.content) # response.choices[0].message.content

def extract_ratings(signal_data):
    results = []
    for signal in signal_data:
        results.append(getRating(signal))
        
    output_file = './smoothness_scores.json'
    with open(output_file, 'w') as f:
        json.dump(results, f)

def extract_keywords(signal_data):
    results = []
    for signal in signal_data:
        results.append(getKeywords(signal))
        
    output_file = './extracted_keywords_GPT_structured.json'
    with open(output_file, 'w') as f:
        json.dump(results, f)

def main():
    extract_ratings(descriptionsJson)

if __name__ == '__main__':
    main()

Extracting ratings for signal index  113
{"signal_id":"F113","smoothness_score":0.4,"smoothness_keywords":["slow heartbeat","slow","heartbeat","comforted","calm","reassuring","buzz notification","enjoyable"],"roughness_keywords":["bumpy-feeling","anxious","greasy","alarm","like a phone vibrating","moments before the results"]}
Extracting ratings for signal index  114
{"signal_id":"F114","smoothness_score":0.2,"smoothness_keywords":["long buzz","continuous","slow continuous vibrations","long in duration","few break-points","pleasant feeling","happy","enjoyable"],"roughness_keywords":["quick vibrations","panicky","fast wave","alarm","surprised","anxious","urgent","annoyed","harsh vibration","panic state","getting late","confusion","alert"]}
Extracting ratings for signal index  115
{"signal_id":"F115","smoothness_score":0.8,"smoothness_keywords":["continuous wave","steady and even","waves of ocean","slowly and steadily","soothing","relaxed feeling","massage","pleasurable","calm","relaxed"